In [34]:
# !pip3 install --upgrade pyiwn

In [14]:
from pyiwn import iwn
import random
import json

In [2]:
#Load wordnet of lang
iwnobj = iwn.IndoWordNet(iwn.Language.KONKANI)

2022-04-13:11:23:41,677 INFO     [iwn.py:43] Loading konkani language synsets...
2022-04-13:11:23:42,105 INFO     [utils.py:157] NumExpr defaulting to 8 threads.


In [3]:
all_verbs = set(iwnobj.all_words(iwn.PosTag.VERB))
all_nouns = set(iwnobj.all_words(iwn.PosTag.NOUN))

In [4]:
list(all_nouns)[:10]

['परपंरावाद',
 'खोलपी',
 'चैतन्य_म्हाप्रभू',
 'पंचायत',
 'संपाती',
 'महापरिमिर्वाण',
 'फ्लावर_फुलावर',
 'दमघोष',
 'निरोगीपण',
 'इंद्रदमन']

In [5]:
len(all_nouns)

26556

In [9]:
# Getting lang freqlist
freqlist_path = "../data/konkani_data/kon.tok.freqlist.txt"
lang_code = "kon"
corpus_words = list([pair.split()[1] for pair in open(freqlist_path, "r").read().split("\n") if len(pair.split())==2])
freqs = list([pair.split()[0] for pair in open(freqlist_path, "r").read().split("\n") if len(pair.split())==2])
frequency = {pair.split()[1]:int(pair.split()[0]) for pair in open(freqlist_path, "r").read().split("\n") if (len(pair.split())==2)}



In [10]:

def get_detractors(related_words, n=10):
    
    detractors = list()
    while len(detractors)!=n:
        detractor = random.choice(freq_words)
        if detractor not in related_words and len(detractor)>2 and len(detractor.split())==1:
            detractors.append(detractor)
    
    return detractors
    
    


def get_qa(word, n=10):
    try:
        synsets = iwnobj.synsets(word)
    except KeyError:
        return None, None
    if len(synsets) == 0:
        return None, None
#     print(synsets)
    all_lemmas = [l for syn in synsets for l in syn.lemma_names() if (l!=word and len(l.split(" "))==1 and len(l)>2 and l in frequency and frequency[l]>MIN_FREQ)]
#     print(all_lemmas)
    if not all_lemmas:
        return None, None
#     print(all_lemmas)
    x = random.randint(0,len(all_lemmas)-1)
    answer = all_lemmas[x]
#     print(answer)
    detractors = get_detractors(all_lemmas+[word], n)
    return (answer, detractors)

In [16]:
# Getting query words 

parameters = [(10, 6), (10, 5), (20, 6), (20, 5), (50, 5)]

for MIN_FREQ, N in parameters:

    query_set = list()
    nouns=0
    verbs=0

    all_words = iwnobj.all_words()
    freq_words = list(filter(lambda x: x in frequency and frequency[x]>MIN_FREQ, all_words))


    for idx, word in enumerate(corpus_words):
    #     if len(query_set)==1000:
    #         break

        if int(freqs[idx])<MIN_FREQ:
            break
        if len(word)<=2:
            continue

        if word not in all_nouns and word not in all_verbs:
            continue

        answer, detractors = get_qa(word, n=N)
        if answer is not None:
            query_set.append((word, answer, detractors))
        if word in all_nouns:
            nouns+=1
        else:
            if word in all_verbs:
                verbs+=1
                
    qa_set = {word:{"answer": answer, "detractors": detractors} for word, answer, detractors in query_set}
    print(query_set[:5])
    print("LENGTH OF TEST, ", len(query_set))
    
    with open("../eval_data/{}_wbst/{}.wbst-{}-{}.json".format(lang_code, lang_code, MIN_FREQ, N), "w") as f:
        json.dump(qa_set, f, ensure_ascii=False, indent=2)




[('हांगा', 'हांगासर', ['भुमिका', 'वर्णन', 'अरबी', 'सदांच', 'रूप', 'पयली']), ('कविता', 'गीत', ['दर्या', 'रितीन', 'तरेन', 'नांव', 'जाण', 'काळ']), ('लोक', 'समाज', ['स्वता', 'उपरांत', 'अधिकार', 'प्रक्रिया', 'उतर', 'मेळून']), ('खूब', 'जायत्यो', ['कुचिपुडी', 'सैमीक', 'उबो', 'देखीक', 'एकलो', 'वांटो']), ('व्हड', 'लांब', ['जाती', 'पावस', 'राजा', 'बरीं', 'प्राथमीक', 'नामना'])]
LENGTH OF TEST,  129
[('हांगा', 'हांगासर', ['आयज', 'लोकांचें', 'लेगीत', 'सुरू', 'वाट']), ('कविता', 'काव्य', ['सत्ता', 'स्पश्ट', 'करप', 'वयल्यान', 'मोग']), ('लोक', 'समाज', ['विचार', 'देवूळ', 'भशेन', 'जाण', 'तेन्ना']), ('खूब', 'साबार', ['दोनूय', 'प्रस्न', 'पांय', 'परिस्थिती', 'बरें']), ('व्हड', 'लांब', ['अर्थीक', 'विचार', 'म्हत्वाचें', 'अध्यक्ष', 'वस्तू'])]
LENGTH OF TEST,  129
[('हांगा', 'हांगासर', ['पयलीं', 'क्रिस्तांव', 'विचार', 'पुराय', 'जरी', 'कमी']), ('खूब', 'कांय', ['वर्णन', 'निर्माण', 'विशीं', 'ल्हान', 'प्रभाव', 'पुस्तक']), ('प्रमाण', 'नदरेन', ['तीन', 'स्वता', 'उदक', 'सत्ता', 'कवी', 'सैमीक']), ('वापर', 'उपेग', ['मुखे

In [17]:
len(qa_set)

1414

[('धेरै', 'असङ्ख्य', ['विवेक', 'दूरसञ्चार', 'देवी', 'सम्भव', 'इन्जिनियर']),
 ('प्राप्त', 'उपलब्ध', ['अभिन्न', 'राष्ट्रीय', 'न्यानो', 'बाकी', 'राजकुमारी']),
 ('कारण', 'मतलब', ['सादा', 'केन्द्र', 'कटहर', 'पुलिस', 'सम्झनु']),
 ('नाम', 'प्रसिद्धि', ['सबैमा', 'क्षेत्र', 'सखरखण्ड', 'पल्ट', 'ढङ्गले']),
 ('जुन', 'चन्द्रमा', ['प्रतिनिधि', 'पुरस्कार', 'अनुबन्धित', 'युद्ध', 'कमल'])]

In [19]:
# Writing dataset
import json
    